In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [2]:
conv1 = nn.Conv2d(1, 6, 5)
conv2 = nn.Conv2d(6, 16, 5)
# an affine operation: y = Wx + b
fc1 = nn.Linear(16 * 5 * 5, 120)
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 10)

In [4]:
x = torch.randn([1, 1, 32, 32])

In [5]:
x = F.max_pool2d(F.relu(conv1(x)), (2, 2))

In [6]:
x

tensor([[[[0.5066, 0.7254, 0.3573,  ..., 0.2383, 1.1275, 0.8354],
          [0.8282, 0.3150, 1.3036,  ..., 1.0157, 1.2733, 0.4106],
          [1.0966, 0.3306, 0.8874,  ..., 1.5110, 1.6439, 0.2394],
          ...,
          [1.0347, 0.2877, 0.5908,  ..., 0.8505, 1.3826, 0.2933],
          [0.8765, 0.7268, 1.0241,  ..., 0.8271, 0.8014, 0.4014],
          [0.8446, 0.4074, 1.1317,  ..., 1.0023, 0.4900, 0.3548]],

         [[0.4141, 0.6549, 0.2815,  ..., 1.4328, 1.0895, 0.2605],
          [1.0525, 0.3710, 1.0641,  ..., 0.7481, 0.6131, 0.4535],
          [0.1461, 1.3274, 0.6011,  ..., 0.6448, 0.4718, 0.2430],
          ...,
          [0.4700, 0.3134, 0.4212,  ..., 0.1921, 0.1743, 0.7534],
          [0.6308, 0.6789, 0.6296,  ..., 0.7273, 0.5046, 0.8699],
          [1.1035, 0.4169, 0.2960,  ..., 0.2455, 1.1797, 0.7887]],

         [[1.0057, 0.3534, 0.9373,  ..., 0.5642, 0.7588, 0.8225],
          [0.2059, 0.3373, 0.5559,  ..., 0.6244, 0.0000, 0.7940],
          [0.6765, 0.4783, 1.0933,  ..., 1

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight


10
torch.Size([6, 1, 5, 5])


In [4]:
_input = torch.randn(1, 1, 32, 32)
out = net(_input)
print(out)

tensor([[-0.1804, -0.0028, -0.0551,  0.0904,  0.0306,  0.1734,  0.1526,  0.0115,
          0.0678, -0.0021]], grad_fn=<ThAddmmBackward>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


tensor(0.5819, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0153, -0.0076, -0.0119, -0.0124,  0.0087, -0.0052])


In [9]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [10]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update